# EXTRACT
# Scraping Women t-shirts from Amazon.es

### 0. Import modules

In [2]:
import pickle
import json

In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')


from selenium import webdriver
PATH = webdriver.FirefoxOptions() 


from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait   # para esperar a que cargue
#from selenium.webdriver.support import expected_conditions as EC   # condiciones esperadas


PATH = webdriver.FirefoxOptions() 

## 1. Use Selenium to scrap the data

### 1.1  Open a Firefox window - driver

In [143]:
#driver.quit()
driver = webdriver.Firefox(options = PATH)
#driver.quit()

### 1.2 Go to: `www.amazon.es`

In [144]:
url = 'https://www.amazon.es/'
driver.get(url)

###  1.3 Accept cookies if asked

In [145]:
try:
    ## if amazon ask for cookies first
    try:
        driver.find_element(By.XPATH, '//*[@id="sp-cc-accept"]').click()

    ## if not, go directly to departments, there amazon will surely ask for cookies
    except:
        driver.find_element(By.XPATH, '/html/body/div/div[1]/div/div[1]/a[2]').click()
        time.sleep(3) # espera 2 segundos
        driver.find_element(By.XPATH, '/html/body/center/table/tbody/tr/td[2]/b[2]/a').click()
        ## cookies
        time.sleep(3) # espera 2 segundos
        driver.find_element(By.XPATH, '//*[@id="sp-cc-accept"]').click()
except:
    print("other new starting page")

### 1.4 Navigate the web "by-clicking" to get `women t-shirts`

hamburguer button > ver todo > ropa > mujer > ropa > camiseta

In [146]:
driver.find_element(By.XPATH, '//*[@id="nav-hamburger-menu"]').click()
time.sleep(4) # espera 2 segundos
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/ul[1]/li[21]/a[1]').click()
time.sleep(4) # espera 2 segundos
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/ul[1]/ul[1]/li[7]/a').click()
time.sleep(4) # espera 2 segundos

#mujer
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/ul[44]/li[4]/a').click() 
time.sleep(4) # espera 2 segundos

#ropa
driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[8]/span/a/span').click()
time.sleep(4) # espera 2 segundos

#camisetas
driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[6]/span/a/span').click()
time.sleep(4) # espera 2 segundos
#driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[6]/span/a/span').click()

# ver todos los resultados
driver.find_element(By.XPATH,'//*[@id="apb-desktop-browse-search-see-all"]').click()
time.sleep(4)


start_path=driver.current_url #save the url of the origin page before selecting colors

### 1.5 Scrap all the content related to the products `by color` and  `by page`


* To move along the colors, I need to click the color, scrap and come back to the starting url. <br/>

* To move along pages within each color, in need to pick a random number between 7-10.  <br/>

* Once I get 20 pages for each of the 16 colors available, the loop ends and saves the data in a dictionary of lists.

* Each time I scrap the data of a page, I need to parse it to text before moving to other urls.

`sel_to_text` function parses selenium object to text and filters out irreleant elements (4 < len < 10) 


In [120]:
def sel_to_text(selenium_list, min_=4, max_=10):
    text_list=[prod.text.split("\n") for prod in selenium_list] # selenium link to text
    text_list = [prod for prod in text_list if min_ < len(prod) < max_] # filter out irrelevant elements
    return text_list

**Don't run it**, super slow code.

In [ ]:
driver.get(start_path)

color_index=range(1,16)

index_list=[7,8,9,10]*30

PAG_COLOR = 20 # define total pages for each color

products_pag_dict = {}

for c in color_index:

    driver.get(start_path)
    color_path=f'/html/body/div[1]/div[1]/div[1]/div[2]/div/div[3]/span/div[1]/div/div/div[8]/ul[4]/span[{str(c)}]/li/span/a'
    test_col=driver.find_elements(By.XPATH, color_path)

    color_value=test_col[0].get_attribute("title")

    color_url=test_col[0].get_attribute("href") # url to the (next) color
    driver.get(color_url)

    print(f"Start with Color: {color_value}")
    time.sleep(5) # espera 5 segundos


    #### # each page of each color - FIRST PAGE
    count=1

    products_pag_list=[] 
    test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
    #test0 = driver.find_elements(By.CLASS_NAME, 'a-section a-spacing-small puis-padding-left-micro puis-padding-right-micro') # GET data 
    TT=sel_to_text(test0)
    products_pag_list.append(TT) # FUN: append the relevant elements into the growing list

        #### # each page of each color - FOLLOWING PAGES
    for index in index_list: # try random "nth-child" indexes

        nextpag=f'a.s-pagination-item:nth-child({str(index)})'

        try: 
            driver.find_element(By.CSS_SELECTOR, nextpag).click()  #CHANGE page /within color X
            time.sleep(4) # espera 4 segundos
            try: 
                test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
                TT=sel_to_text(test0)
                products_pag_list.append(TT) # FUN: append the relevant elements into the growing list

                time.sleep(2) # espera 2 segundos
                    #
                count += 1
                print("y", index)

            except:
                print("-", index)    
        except:
            print("n", index)
            #pass

        if count <= PAG_COLOR :
            print(f"Count: {count},  Continue {color_value}") #############

        else: # get info from 20 pages - STOP and GO to other color
            products_pag_dict[color_value] = products_pag_list # add the color list to a dict

            print("Color done")
            driver.get(start_path) # back to starting link to select a new color
            time.sleep(5) # espera 5 segundos
            break
            print("shouldn't be printed")

    print(f"Count: {count},  Next color")

print("All done!!")    

## 3 Save raw data - using pickle

**Don't run it**  to avoid overwritting!

In [205]:
#amz1_womentshirt_data=products_pag_dict

#with open('amz1_womentshirt_data.pkl', 'wb') as file:
#    pickle.dump(amz1_womentshirt_data, file)


### Extra. Security copy - scrap data just for `negro`

In [ ]:


driver.get(start_path)
color_index=1
index_list=[7,8,9,10]*2
PAG_COLOR = 2 # define total pages for each color

products_pag_dict = {}


# start_path=driver.current_url #save the url of the origin page before selecting colors


color_path=f'/html/body/div[1]/div[1]/div[1]/div[2]/div/div[3]/span/div[1]/div/div/div[8]/ul[4]/span[{str(c)}]/li/span/a'
test_col=driver.find_elements(By.XPATH, color_path)
    
color_value=test_col[0].get_attribute("title")
    
color_url=test_col[0].get_attribute("href") # url to the (next) color
driver.get(color_url)
    
print(f"Start with Color: {color_value}")
time.sleep(5) # espera 5 segundos

    
    
#### # each page of each color - FIRST PAGE
count=1
    
products_pag_list=[] 
test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
#test0 = driver.find_elements(By.CLASS_NAME, 'a-section a-spacing-small puis-padding-left-micro puis-padding-right-micro') # GET data 
TT=sel_to_text(test0)
products_pag_list.append(TT) # FUN: append the relevant elements into the growing list
   
    #### # each page of each color - FOLLOWING PAGES
for index in index_list: # try random "nth-child" indexes

    nextpag=f'a.s-pagination-item:nth-child({str(index)})'
        
    try: 
        driver.find_element(By.CSS_SELECTOR, nextpag).click()  #CHANGE page /within color X
        time.sleep(4) # espera 4 segundos
        try: 
            test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
            TT=sel_to_text(test0)
            products_pag_list.append(TT) # FUN: append the relevant elements into the growing list

            time.sleep(2) # espera 2 segundos
                #
            count += 1
            print("y", index)
        
        except:
            print("-", index)    
    except:
        print("n", index)
        #pass
    
    if count <= PAG_COLOR :
        print(f"Count: {count},  Continue {color_value}") #############
        
    else: # get info from 20 pages - STOP and GO to other color
        products_pag_dict[color_value] = products_pag_list # add the color list to a dict
            
        print(f"Count: {count},  Next color")
        driver.get(start_path) # back to starting link to select a new color
        time.sleep(5) # espera 5 segundos
        break
        print("shouldnt be printed!!")
        


print("done!!")    